In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

##### Load data from 293T, MEF lenti infections (`data_lenti`)

In [ ]:
base_path_1 = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.04.05_exp89'/'export'
base_path_2 = rd.datadir/'instruments'/'data'/'attune'/'chris'/'2024.06.02-exp95-lenti-miR-iFFL'/'export'
plate_list = ['_'.join(x) for x in zip(
        ['plate'+str(i) for i in range(1,10)], 
        (['293T']*3 + ['MEF2A']*3 + ['MEF8A']*3),
        ['P9','P14','P15']*3
    )]

plates = pd.DataFrame({
    'data_path': [base_path_1/'293T_control', 
                  base_path_1/'293T_plate1', base_path_1/'293T_plate2', base_path_1/'293T_plate3',
                  base_path_1/'MEF_3_plate1', 
                  base_path_1/'MEF_4-1_plate1', base_path_1/'MEF_4-1_plate2', base_path_1/'MEF_4-1_plate3'] +
                 [base_path_2/p for p in plate_list],
    'yaml_path': [base_path_1/'kasey_yaml2'/'plate_control.yaml', 
                  base_path_1/'kasey_yaml2'/'plate01.yaml', base_path_1/'kasey_yaml2'/'plate02.yaml', base_path_1/'kasey_yaml2'/'plate03.yaml',
                  base_path_1/'kasey_yaml2'/'mef_3_plate01.yaml', 
                  base_path_1/'kasey_yaml2'/'mef_4-1_plate01.yaml', base_path_1/'kasey_yaml2'/'mef_4-1_plate02.yaml', base_path_1/'kasey_yaml2'/'mef_4-1_plate03.yaml'] +
                 [base_path_2/(p+'_metadata.yaml') for p in plate_list],
})

cache_path = rd.rootdir/'output'/'fig_designs'/'data_lenti.gzip'
output_path = rd.rootdir/'output'/'fig_applications'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx'

# Load data
data_lenti = pd.DataFrame()
if cache_path.is_file(): data_lenti = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data_lenti = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_lenti = data_lenti[data_lenti[c]>0]
    
    data_lenti.dropna(inplace=True)
    data_lenti.to_parquet(rd.outfile(cache_path))

data_lenti['cell'] = data_lenti['cell_type'].apply(lambda x: x.split('-')[0])
data_lenti['exp'] = data_lenti['cell_type'] + '_' + data_lenti['virus_batch']
data_lenti['moi'] = data_lenti['virus_dilution']

def map_biorep(df):
    biorep_map = {val:i for i,val in enumerate(df['exp'].unique())}
    d = df.copy()
    d['biorep'] = d['exp'].map(biorep_map)
    return d

data_lenti = data_lenti.groupby('cell')[data_lenti.columns].apply(map_biorep).reset_index(drop=True)
display(data_lenti)

In [ ]:
# Gate cells
gates_lenti = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates_lenti[channel] = data_lenti[(data_lenti['virus_dilution']==0)].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_lenti.reset_index(inplace=True)
display(gates_lenti)

# Add missing gates
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P10'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P14_'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P16'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values) 
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-3_P10'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P10'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P14'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P16'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)

# Indicate which channels are relevant for each experiment
gates_lenti.sort_values(['exp'], inplace=True)
gates_lenti['marker'] = 'mGL-A'
gates_lenti['output'] = 'mRuby2-A'

# Gate data by marker expression
data_lenti = data_lenti.groupby(['cell_type','virus_batch'])[data_lenti.columns].apply(lambda x: base.gate_data(x,gates_lenti))
data_lenti.reset_index(inplace=True, drop=True)
df_lenti = data_lenti[(data_lenti['expressing']) & (data_lenti['virus_dilution']!=0)]

##### Load data from lenti infection of T cells (`data_tcell`)

In [ ]:
# Load T cell lenti data
tcell_path = rd.datadir/'instruments'/'data'/'collaborators'/'birnbaum_steph'/'2024-06-10 Galloway Exp 1'
cache_path = output_path/'data_tcell.gzip'

data_tcell = pd.DataFrame()
if cache_path.is_file(): data_tcell = pd.read_parquet(cache_path)
else: 
    channels = ['FSC-A', 'SSC-A', 'FITC-A', 'PE-A', 'APC-A750-A']
    data_tcell = rd.flow.load_csv_with_metadata(tcell_path/'export', tcell_path/'metadata.yaml', columns=channels)

    # Remove negative channel values
    for c in channels: data_tcell = data_tcell[data_tcell[c]>0]

    data_tcell = data_tcell.rename({'FITC-A': 'mGL-A', 'PE-A': 'mRuby2-A', 'APC-A750-A': 'livedead-A'}, axis=1)
    data_tcell.to_parquet(rd.outfile(cache_path))

data_tcell['exp'] = 'steph'
data_tcell['biorep'] = 1
data_tcell['cell'] = 'tcell'
display(data_tcell)

In [ ]:
# Gate cells
gates_tcell = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates_tcell[channel] = data_tcell[data_tcell['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.99995)) # adjust gate to better isolate marker+ pop
gates_tcell.reset_index(inplace=True)
display(gates_tcell)

# Indicate which channels are relevant for each experiment
gates_tcell.sort_values(['exp'], inplace=True)
gates_tcell['marker'] = 'mGL-A'
gates_tcell['output'] = 'mRuby2-A'

# Gate data by marker expression and live/dead stain
data_tcell = data_tcell.groupby('exp')[data_tcell.columns].apply(lambda x: base.gate_data(x,gates_tcell))
data_tcell.reset_index(inplace=True, drop=True)
livedead_gate = 2000
df_tcell = data_tcell[(data_tcell['expressing']) & (data_tcell['construct']!='UT') & (data_tcell['livedead-A']<livedead_gate)]

##### Load rat cortical neuron lenti infection (`data_neuron`)

In [ ]:
# Load data
neuron_path = rd.datadir/'instruments'/'data'/'attune'/'chris'/'2024.06.15-rat-neurons'
cache_path = output_path/'data_neuron.gzip'

data_neuron = pd.DataFrame()
if cache_path.is_file(): data_neuron = pd.read_parquet(cache_path)
else: 
    channels = ['FSC-A', 'SSC-A', 'mGL-A', 'mRuby2-A', 'mCherry-A']
    data_neuron = rd.flow.load_csv_with_metadata(neuron_path/'export', neuron_path/'metadata.yaml', columns=channels)

    # Remove negative channel values
    for c in channel_list: data_neuron = data_neuron[data_neuron[c]>0]
    
    data_neuron.to_parquet(rd.outfile(cache_path))

data_neuron['biorep'] = 1
data_neuron['exp'] = 'cpj'
data_neuron['cell'] = 'neuron'
data_neuron['dox'] = 1000
display(data_neuron)

In [ ]:
# Gate cells
gates_neuron = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates_neuron[channel] = data_neuron[data_neuron['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_neuron.reset_index(inplace=True)
display(gates_neuron)

# Indicate which channels are relevant for each experiment
gates_neuron.sort_values(['exp'], inplace=True)
gates_neuron['marker'] = 'mGL-A'
gates_neuron['output'] = 'mRuby2-A'

# Adjust marker gate to better isolate infected population
gates_neuron['mGL-A'] = 1e3

# Gate data by marker expression
data_neuron = data_neuron.groupby('exp')[data_neuron.columns].apply(lambda x: base.gate_data(x,gates_neuron))
data_neuron.reset_index(inplace=True, drop=True)
df_neuron = data_neuron[(data_neuron['expressing']) & (data_neuron['construct']!='UT')]

# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df_neuron = data_neuron[(data_neuron['construct']=='UT')].groupby(['exp'])['output'].apply(sp.stats.gmean).reset_index()

##### Load iPSC data (`data_ipsc`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
exp83_5_path = base_path/'2024.04.29_exp83.5'/'export'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx'

# Kasey data
plates = pd.DataFrame({
    'data_path': [exp83_5_path],
    
    'yaml_path': [exp83_5_path/'exp83.5_wells.yaml'],
    
    'biorep': [1],
    
    'exp': ['exp83.5']
})

cache_path = rd.rootdir/'output'/'fig_designs'/'data_ipsc.gzip'
output_path = rd.rootdir/'output'/'fig_applications'

# Load data
data_ipsc = pd.DataFrame()
if cache_path.is_file(): data_ipsc = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data_ipsc = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_ipsc = data_ipsc[data_ipsc[c]>0]
    
    data_ipsc.dropna(inplace=True)
    data_ipsc.to_parquet(rd.outfile(cache_path))

In [ ]:
# Emma data (YL1)
elp_path = rd.datadir/'instruments'/'data'/'attune'/'Emma'

plates = pd.DataFrame({
    'data_path': [elp_path/'2024.06.14_EXP65'/'data_singlets', elp_path/'2024.06.21_EXP65.2'/'data_singlets'],
    
    'yaml_path': [exp83_5_path/'exp83.5_wells.yaml']*2,
    
    'biorep': [2,3],
    
    'exp': ['elp_exp65', 'elp_exp65.2']
})

cache_path = rd.rootdir/'output'/'fig_applications'/'data_ipsc.gzip'
output_path = rd.rootdir/'output'/'fig_applications'

# Load data
data_ipsc2 = pd.DataFrame()
if cache_path.is_file(): data_ipsc2 = pd.read_parquet(cache_path)
else: 
    channel_list = ['YL1-A','mRuby2-A','FSC-A','SSC-A','mGL-A']
    data_ipsc2 = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_ipsc2 = data_ipsc2[data_ipsc2[c]>0]
    data_ipsc2.dropna(inplace=True)

    # Rename YL1 to mRuby2 (rename YL2 to mCherry to match Kasey's data)
    data_ipsc2.rename(columns={'YL1-A': 'mRuby2-A', 'mRuby2-A': 'mCherry-A'}, inplace=True)

    # Merge with other data
    data_ipsc = pd.concat([data_ipsc, data_ipsc2], ignore_index=True)
    data_ipsc.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(metadata_path, style='designs')
data_ipsc = data_ipsc.merge(metadata, how='left', on='construct')
display(data_ipsc)

In [ ]:
# Gate cells
gates_ipsc = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates_ipsc[channel] = data_ipsc[data_ipsc['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_ipsc.reset_index(inplace=True)
display(gates_ipsc)

# Indicate which channels are relevant for each experiment
gates_ipsc.sort_values(['exp'], inplace=True)
gates_ipsc['marker'] = 'mGL-A'
gates_ipsc['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data_ipsc = data_ipsc.groupby('exp')[data_ipsc.columns].apply(lambda x: base.gate_data(x,gates_ipsc))
data_ipsc.reset_index(inplace=True, drop=True)
df_ipsc = data_ipsc[(data_ipsc['expressing']) & (data_ipsc['construct']!='UT')]

In [ ]:
# Load iPSC lenti infection data
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
plates = pd.DataFrame({
    'data_path': [base_path/'2024.10.03_exp116'/'export'/'plate2', base_path/'2024.10.21_exp116.2'/'export'],
    'yaml_path': [base_path/'2024.10.03_exp116'/'export'/'plate2'/'exp116_wells.yaml', base_path/'2024.10.21_exp116.2'/'export'/'wells.yaml'],
})
output_path = rd.rootdir/'output'/'lenti_iPS11'
cache_path = output_path/'lenti_iPS11.gzip'

# Load data
data_ipsc_lenti = pd.DataFrame()
if cache_path.is_file(): data_ipsc_lenti = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','tagBFP-A','mGL-A']
    data_ipsc_lenti = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_ipsc_lenti = data_ipsc_lenti[data_ipsc_lenti[c]>0]
    
    data_ipsc_lenti.dropna(inplace=True)
    data_ipsc_lenti.to_parquet(rd.outfile(cache_path))

data_ipsc_lenti['marker'] = data_ipsc_lenti['mGL-A']
data_ipsc_lenti['output'] = data_ipsc_lenti['mRuby2-A']
data_ipsc_lenti['exp'] = 'exp116_' + data_ipsc_lenti['biorep'].astype(str)

# Add metadata for constructs
data_ipsc_lenti = data_ipsc_lenti.merge(metadata, how='left', on='construct')

data_ipsc_lenti.dropna(inplace=True)
display(data_ipsc_lenti)

In [ ]:
# Gate cells
gates_ipsc_lenti = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A',]
for channel in channel_list:
    gates_ipsc_lenti[channel] = data_ipsc_lenti[data_ipsc_lenti['construct']=='UI'].groupby(['biorep'])[channel].apply(lambda x: x.quantile(0.999))
gates_ipsc_lenti.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates_ipsc_lenti['marker'] = 'mGL-A'
gates_ipsc_lenti['output'] = 'mRuby2-A'
gates_ipsc_lenti['exp'] = 'exp116_' + gates_ipsc_lenti['biorep'].astype(str)

# Adjust marker gate to be more stringent
gates_ipsc_lenti['mGL-A'] = [2e3]*3
display(gates_ipsc_lenti)

# Gate data by transfection marker expression
data_ipsc_lenti = data_ipsc_lenti.groupby('exp')[data_ipsc_lenti.columns].apply(lambda x: base.gate_data(x,gates_ipsc_lenti))
data_ipsc_lenti.reset_index(inplace=True, drop=True)
df_ipsc_lenti = data_ipsc_lenti[(data_ipsc_lenti['expressing']) & (data_ipsc_lenti['construct']!='UI')]

##### Load therapeutic neuro genes transfection (`data_therapeutic`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'Emma'
rep1_2_path = base_path/'2024.06.05_EXP56'/'data_singlets'
rep3_path = base_path/'2024.06.09_EXP60'/'data_singlets'

plates = pd.DataFrame({
    'data_path': [rep1_2_path, rep1_2_path, rep3_path],
    'yaml_path': [rep1_2_path/'elp_exp56_biorep_1_wells.yaml', rep1_2_path/'elp_exp56_biorep_2_wells.yaml', rep3_path/'elp_exp56_biorep_3_wells.yaml'],
    'exp': ['elp_exp56.1','elp_exp56.2','elp_exp60'],
    'biorep': [1,2,3]
})

cache_path = output_path/'data_therapeutics.gzip'

# Load data
data_therapeutics = pd.DataFrame()
if cache_path.is_file(): data_therapeutics = pd.read_parquet(cache_path)
else: 
    channel_list = ['mRuby2-A','EGFP-A','iRFP-A']
    data_therapeutics = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_therapeutics = data_therapeutics[data_therapeutics[c]>0]
    
    data_therapeutics.dropna(inplace=True)
    data_therapeutics.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(metadata_path, style='designs')
data_therapeutics = data_therapeutics.merge(metadata, how='left', on='construct')

In [ ]:
# Gate cells
gates_therapeutics = pd.DataFrame()
channel_list = ['EGFP-A', 'mRuby2-A', 'iRFP-A']
for channel in channel_list:
    gates_therapeutics[channel] = data_therapeutics[data_therapeutics['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_therapeutics.reset_index(inplace=True)
display(gates_therapeutics)

# Indicate which channels are relevant for each experiment
gates_therapeutics['marker'] = 'iRFP-A'
gates_therapeutics['output'] = 'mRuby2-A'

# Gate data by marker expression
data_therapeutics = data_therapeutics.groupby('exp')[data_therapeutics.columns].apply(lambda x: base.gate_data(x,gates_therapeutics))
data_therapeutics.reset_index(inplace=True, drop=True)
data_therapeutics.loc[data_therapeutics['name'].str.contains('FMRP'), 'output'] = data_therapeutics.loc[data_therapeutics['name'].str.contains('FMRP'), 'EGFP-A']
df_therapeutics = data_therapeutics[(data_therapeutics['expressing']) & (data_therapeutics['construct']!='UT')]

##### Metadata, color palettes, etc.

In [ ]:
# Define metadata
metadata = base.get_metadata(metadata_path, style='designs')
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

# Create dicts to specify colors/markers
metadata2 = base.get_metadata(metadata_path, style='applications')
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
app_palette = metadata_dict2['color']
app_markers = metadata_dict2['markers']

##### Combine & compute stats

In [ ]:
# Combine lenti data into a single df (data_comb)
data_comb = pd.concat([df_lenti, df_tcell, df_neuron], ignore_index=True)
data_comb.dropna(axis='columns', inplace=True)
display(data_comb)

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data_lenti[(data_lenti['virus_dilution']==0)].groupby(['cell','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()
baseline_df_tcell = data_tcell[(data_tcell['construct']=='UT')].groupby(['cell','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()
baseline_df_neuron = data_neuron[(data_neuron['construct']=='UT')].groupby(['cell','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()
baseline_df = pd.concat([baseline_df, baseline_df_tcell, baseline_df_neuron])
display(baseline_df)

In [ ]:
# Bin data and calculate statistics
df_quantiles, stats, stats_quantiles, fits = base.calculate_bins_stats(data_comb, by=['cell','exp','biorep','construct','moi','dox'], num_bins=20, stat_list=[sp.stats.gmean, np.std, sp.stats.variation])
df_quantiles_fewer_bins, _, _, _ = base.calculate_bins_stats(data_comb, by=['cell','exp','biorep','construct','moi','dox'], num_bins=10)
df_quantiles_ipsc, stats_ipsc, stats_quantiles_ipsc, fits_ipsc = base.calculate_bins_stats(df_ipsc, by=['exp','biorep','construct'], num_bins=20)
df_quantiles_therapeutics, stats_therapeutics, _, fits_therapeutics = base.calculate_bins_stats(df_therapeutics, by=['exp','biorep','construct'], num_bins=20)

# Add metadata
df_quantiles = df_quantiles.merge(metadata, how='left', on='construct')
stats = stats.merge(metadata, how='left', on='construct')
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
fits = fits.merge(metadata, how='left', on='construct')
df_quantiles_fewer_bins = df_quantiles_fewer_bins.merge(metadata, how='left', on='construct')

stats_ipsc = stats_ipsc.merge(metadata, how='left', on='construct')
fits_ipsc = fits_ipsc.merge(metadata, how='left', on='construct')

stats_therapeutics = stats_therapeutics.merge(metadata, how='left', on='construct')
fits_therapeutics = fits_therapeutics.merge(metadata, how='left', on='construct')

In [ ]:
# (TEMP) stats for iPSC data only
df_quantiles_ipsc, stats_ipsc, stats_quantiles_ipsc, fits_ipsc = base.calculate_bins_stats(df_ipsc, by=['exp','biorep','construct'], num_bins=20)
stats_ipsc = stats_ipsc.merge(metadata, how='left', on='construct')
fits_ipsc = fits_ipsc.merge(metadata, how='left', on='construct')

df_quantiles_ipsc_lenti, stats_ipsc_lenti, stats_quantiles_ipsc_lenti, fits_ipsc_lenti = base.calculate_bins_stats(df_ipsc_lenti, by=['exp','biorep','construct'], num_bins=20)
stats_ipsc_lenti = stats_ipsc_lenti.merge(metadata, how='left', on='construct')
fits_ipsc_lenti = fits_ipsc_lenti.merge(metadata, how='left', on='construct')

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6, 1+1.5*4)) #4.48819, 6.8504
fig_gridspec = matplotlib.gridspec.GridSpec(5, 6, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[1]+[1.5]*4, width_ratios=[1]*6)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,:3]),
    'C': fig.add_subfigure(fig_gridspec[1,3:]),
    'D': fig.add_subfigure(fig_gridspec[2,:3]),
    'E': fig.add_subfigure(fig_gridspec[2,3:]),
    'F': fig.add_subfigure(fig_gridspec[3,:3]),
    'G': fig.add_subfigure(fig_gridspec[3,3:]),
    'H': fig.add_subfigure(fig_gridspec[4,:2]),
    'I': fig.add_subfigure(fig_gridspec[4,2:4]),
    'I2': fig.add_subfigure(fig_gridspec[4,4:]),
}
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.1, linewidth=0.5, edgecolor='white')

fig_name = 'fig_applications.pdf'
fig_path = rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'
fig.savefig(rd.outfile(output_path/fig_name))   # save locally
fig.savefig(rd.outfile(fig_path/fig_name))      # save to OneDrive

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = df_quantiles_fewer_bins[(df_quantiles_fewer_bins['dox']==1000) & (df_quantiles_fewer_bins['moi']==5) & 
                                  (df_quantiles_fewer_bins['cell']=='neuron')].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = df_quantiles[(df_quantiles['group'].isin(['controller','base'])) & (df_quantiles['dox']==1000) & 
                        (df_quantiles['moi']==1) & (df_quantiles['cell']=='MEF') & (df_quantiles['design']<=1) &
                        (((df_quantiles['ts_kind']=='na') & (df_quantiles['biorep']==6)) | ((df_quantiles['biorep']==1)))
                      ].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(1e3,2e5))
sns.despine(ax=ax)
baseline = baseline_df.loc[baseline_df['cell']=='MEF', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = df_quantiles[(df_quantiles['group'].isin(['controller','base'])) & (df_quantiles['dox']==1000) & 
                        (df_quantiles['moi']==1) & (df_quantiles['cell']=='tcell') &
                        (df_quantiles['design']<=1)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker')
sns.despine(ax=ax)
ax.yaxis.set_tick_params(which='minor', left=False)

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.5, bottom=0.45, top=0.2, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = df_quantiles_ipsc_lenti[(df_quantiles_ipsc_lenti['group'].isin(['controller','base']))  &
                            (df_quantiles_ipsc_lenti['design']<=1) & (df_quantiles_ipsc_lenti['biorep']==1)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)
baseline = stats_ipsc.loc[stats_ipsc['group']=='marker', 'output_gmean'].mean()
#ax.axhline(baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.35, top=0.3, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = df_quantiles_therapeutics[(df_quantiles_therapeutics['group'].isin(['controller','base'])) &
                        (df_quantiles_therapeutics['design']<=1) & (df_quantiles_therapeutics['biorep']==1) &
                        (df_quantiles_therapeutics['name'].str.contains('FXN'))].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=app_palette, 
             legend=False, dashes=False, style='construct', markers=app_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker', title='FXN')
sns.despine(ax=ax)
baseline = stats_therapeutics.loc[stats_therapeutics['group']=='marker', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=app_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))

In [ ]:
subfig = subfigures['I2']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.35, top=0.3, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3))

plot_df = df_quantiles_therapeutics[(df_quantiles_therapeutics['group'].isin(['controller','base'])) &
                        (df_quantiles_therapeutics['design']<=1) & (df_quantiles_therapeutics['biorep']==1) &
                        (df_quantiles_therapeutics['name'].str.contains('FMRP'))].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=app_palette, 
             legend=False, dashes=False, style='construct', markers=app_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             hue_order=plot_order)
ax.set(xscale='log', yscale='log', xlabel='marker', title='FMRP1')
sns.despine(ax=ax)
baseline = stats_therapeutics.loc[stats_therapeutics['group']=='marker', 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=app_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.45, top=0.3, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1.75), wspace=0.5))

# slope
ax = axes[0]
plot_df = fits[(fits['cell']=='neuron') & (fits['moi']==1)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=app_palette,
                  legend=False, ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0,1.2))
rd.plot.generate_xticklabels(metadata, 'construct', ['ts_label'], ax=ax, annotate=False)
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# CV
ax = axes[1]
plot_df = stats[(stats['cell']=='neuron') & (stats['moi']==1)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_variation', hue='construct', palette=app_palette,
                  legend=False, ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='', ylim=(0.3,1))
rd.plot.generate_xticklabels(metadata, 'construct', ['ts_label'], ax=ax, annotate=False)
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# CDF
ax = axes[2]
plot_df = df_quantiles[(df_quantiles['cell']=='neuron') & (df_quantiles['moi']==1)]

sns.kdeplot(data=plot_df, x='output', hue='construct', palette=main_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(2e1,2e4), title='CDF')
sns.despine(ax=ax)
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path/fig_name))
fig.savefig(rd.outfile(fig_path/fig_name))